# Robust Facility Location (WORK IN PROGRESS)

The Facility Location Problem (FLP) is a fundamental optimization challenge focusing on determining the optimal locations for facilities such as warehouses, plants, or distribution centers to minimize costs or maximize service coverage. It involves deciding where to place these facilities to efficiently serve demands from customers or locations, considering factors like transportation costs, facility setup costs, and capacity constraints. The goal of the facility location problem is to find the best allocation that satisfies demands and maxinimize profits.

Let $ T, F, N \in \mathbb{N}$ be the length of the horizon, the number of candidate locations to which a facility can be assigned, and the number of locations that have a demand for the facility respectively. Let $\eta \in \mathbb{R}_+ $ be the unit price of goods. Let $\vec{c}_{i}, \vec{C}_{i} ,\vec{K}_{i} \in \mathbb{R}_+ $ be the unit cost of production, cost of capacity and the cost of opening a location at facility $i$ respectively. Let $\vec{d}_{ij} \in \mathbb{R}_+ $ be the cost of shipping from location $i$ to location $j$. Let $\vec{D}_{j\tau} $ be the demand for period $\tau$ at location $j$. Decision variable $\vec{x}_{ij\tau}$ represents the proportion of the demand at location $j$ during period $\tau$ that is satisfied by facility $i$. $\vec{P}_{i\tau}$ represents the amount of good that is produced at facility $i$ during the period $\tau$. For each facility $i$, the decision variable $\vec{I}_i$ denotes whether the facility in location $i $ is open or closed, by taking values 1 or 0, respectively, and $Z_i$ denotes the capacity of the facility in this location in case it is open.
$M$ is a large constant.


With this information in mind, our optimization problem becomes:
$$
\begin{aligned}
& \text{maximize} \quad (\eta \cdot \vec{1}^T - \vec{d}) \cdot (\vec{x} \cdot \vec{D}) -  \vec{c}^T (1^T\vec{P}) - \vec{C}^T\vec{Z} - \vec{K}^T\vec{Z} \\
& \text{subject to} \\
& \vec{1}^T \vec{x}_{j\tau} \leq 1, \\
&  \vec{x}_{i\tau}^T \vec{D}_{\tau} \leq P_{i\tau}, \\
& x_{ij\tau} \geq 0,  \\
&  P_{i\tau} \leq Z_i, \\
&  Z_i \leq M I_i
\end{aligned}
$$

Lets look at each term of the objective function:

- The expression $\sum_{\tau = 1}^T \sum_{i = 1}^F \sum_{j = 1}^N (\eta - d_{ij}) x_{ij\tau} D_{i\tau} $ represents the revenue generated from satisfying demand at location  $i$, time period $ \tau $, and serving demand from location $ j$. It calculates the difference between the revenue $ \eta $ and the unit demand $ d_{ij} $, multiplied by the decision variable $x_{ij\tau} $ and the nominal demand $ D_{i\tau} $.

- The term $ \sum_{\tau = 1}^T \sum_{i = 1}^F c_i P_{i\tau} $ represents the total production cost incurred at facility $ i $ across all time periods $ \tau $. It accounts for the production cost $ c_i $ associated with producing $ P_{i\tau} $ units at facility $ i $.

- The expression $ \sum_{i = 1}^F C_i Z_i $ represents the fixed costs associated with opening facility $ i $, where $ C_i $ denotes the fixed cost and $ Z_i $ represents the binary decision variable indicating whether facility $ i $ is open $( Z_i = 1 )$ or closed $(Z_i = 0)$.

- Finally, $ \sum_{i = 1}^F K_i I_i $ represents the penalty or cost incurred if facility $ i $ remains open but does not operate efficiently. Here, $ K_i $ represents the penalty cost associated with inefficiency at facility $ i $, and $ I_i $ is a binary variable indicating whether facility $ i $ operates inefficiently $( I_i = 1 )$ or efficiently $( I_i = 0 )$.

In the robust version of this problem, we assume that the demand $d$ is uncertain. The parameter belongs to the Ellipsoidal Uncertainty set, which can be reprsented like this: 

$$ \mathcal{U} = \left\{ D \in \mathbb{R}^{N \times T} \ \middle|\  \left\| \frac{D - \bar{D}}{\epsilon \cdot \bar{D}} \right\|_F^2 \leq \rho^2 \right\}
$$
In this equation, $\vec{D}$ is the vector of demands. $\vec{\bar{D}}$ is the vector of nominal demands. $\vec{\epsilon} $ is the vector of parameters related to the variability of demands. $\vec{\rho}$ is the size of the ellipsoidal.


Our optimization problem can thus be converted into the following robust counterpart:

$$
\begin{aligned}
& \text{maximize} \quad \theta \\
& \text{subject to} \\
& ( \vec{1}^T \eta- \vec{d}) \cdot (\vec{x} \cdot \vec{D}) - \rho \| Q(x) \|_2 - \vec{c}^T (1^T\vec{P}) - \vec{C}^T\vec{Z} - \vec{K}^T\vec{Z} \geq \theta, \\
& \vec{x}_{i\tau}^T \vec{D}_{\tau} + \rho \| V_{i\tau} \|_2 \leq P_{i\tau}, \\
& \vec{1}^T \vec{x}_{j\tau} \leq 1, \\
& x_{ij\tau} \geq 0,  \\
&  P_{i\tau} \leq Z_i, \\
& Z_i \leq M I_i
\end{aligned}
$$

where $ Q_{jt}(x) = (\vec{\eta} - \vec{d}_j)^T \vec{x}_{j\tau}  \vec{\epsilon}_t \vec{\bar{D}}_{\tau} $

An important point to consider is that this robust counterpart does not directly apply an uncertain parameter in the equations. However, it is a conic quadratic problem that is the equivalent of the original robust problem. It is also particularely useful in these situations are LROPT does not support $2$ - dimensional uncertain parameters at the time of the creation of this example. 

In [1]:
import numpy as np 
import cvxpy as cp 
import lropt

Next, we will generate the data. We will consider an example where we have $5$ facilities and $8$ candidate locations. The length of each horizon is $10$. Finally, the unit price of each good is $100$.

In [2]:
#Keeping comments until this is finalized
T = 10  # Length of the horizon
F = 5   # Number of facilities
N = 8  # Number of candidate locations
np.random.seed(1)

eta = 100.0  # Unit price of goods
rho = 100
M = 1100
c = np.random.rand(F)  # Unit cost of production for each facility
C = np.random.rand(F)  # Cost of capacity for each facility
K = np.random.rand(F)  # Cost of opening a facility for each facility
d = np.random.rand(F, N)  # Cost of shipping from facility i to location j
D = np.random.rand(N, T)  # Demand at each location and time period
D_normalized = (D - np.min(D)) / (np.max(D) - np.min(D))

NUM_DEC = 3
x = cp.Variable((F*T,N), nonneg = True)   #Flattened x
P = cp.Variable((F, T), nonneg=True)     # Production amount at each facility and time
Z = cp.Variable(F, boolean=True)         # Binary variable indicating whether facility i is open
I = cp.Variable(F, boolean=True)         # Binary variable indicating whether facility i is open
epsilon = np.random.rand(T)
theta = cp.Variable()


We will define these variables to help us in formulating constraints and simplifiying our calculations.

In [3]:
f1 = np.tile(eta - d, (T, 1)).flatten()
f2 = np.tile(D, (F ,1)).flatten()
f3 = np.tile(D_normalized, (F ,1)).flatten()

We will then define our constraints.

In [4]:
constraints = []

expr = cp.sum(cp.multiply(f1, x.flatten()) @ f3)
norm_expr = cp.norm(expr)

revenue = cp.sum(cp.multiply((f1), x.flatten()) @ f2) + cp.multiply(rho, norm_expr.is_affine())
cost_production = cp.sum(c @ P)
fixed_costs = cp.sum(cp.multiply(C, Z))
penalties = cp.sum(cp.multiply(K, I))

constraints.append(revenue - cost_production - fixed_costs - penalties >= theta)
constraints.append(cp.sum(x.flatten()) <= 1)
constraints.append(x.flatten()>=0)

Here, we use a helper function to simplify calculations.

In [5]:
def idx(t, f, n, T, F, N):
    return t * (F * N) + f * N + n
for i in range(F):
    for tau in range(T):
        V_i_tau = cp.sum(cp.multiply(epsilon[tau] * x[i*T + tau, :], D[:, tau]))
        constraint = cp.sum(cp.multiply(x[i*T + tau, :], D[:, tau])) + rho * cp.sum_squares(V_i_tau) <= P[i, tau]
        constraints.append(constraint)

P_T = P.T
for t in range(T):
    constraints.append(P_T[t]<=Z)

constraints.append(Z <= M*I)

Finally, we can define the objective and get the optimal value for the equation. 

In [6]:
objective = cp.Maximize(theta)
prob = cp.Problem(objective, constraints)
sol = prob.solve()
print(f"The robust optimal value using  is {round(prob.solve(), NUM_DEC)}") # change it to 2 decimal places

The robust optimal value using  is 96.897
